## New Built-In Projections

ipyleaflet now supports custom map projections and includes base layers for polar projections:
- NASA's Next Generation Blue Marble 500m for the Arctic and Antarctic regions
- Esri Arctic Ocean Basemap and Antarctic Basemap

In [ ]:
from ipyleaflet import (
    Map,
    basemaps,
    basemap_to_tiles,
    Marker,
    TileLayer,
    WMSLayer,
    Polygon,
    GeoJSON,
    DrawControl,
    projections,
)


def handle_draw(target, action, geo_json):
    print(action, geo_json)


dc = DrawControl(marker={"shapeOptions": {"color": "#0000FF"}})
dc.on_draw(handle_draw)

# note that we need to use the same projection for the layer and the map.
m1 = Map(
    center=(90, 0),
    zoom=0,
    basemap=basemaps.NASAGIBS.BlueMarble3413,
    crs=projections.EPSG3413.NASAGIBS,
)
m1.add(dc)
m1

In [ ]:
# note that we need to use the same projection for the layer and the map.
m2 = Map(
    center=(-90, 0),
    zoom=0,
    basemap=basemaps.Esri.AntarcticBasemap,
    crs=projections.EPSG3031.ESRIBasemap,
)

# add draw control on Antarctic map
dc2 = DrawControl(marker={"shapeOptions": {"color": "#0000FF"}})
dc2.on_draw(handle_draw)
m2.add(dc2)

# MODIS Mosaic of Antarctica (MOA)
MOA3031 = dict(
    name="EPSG:3031",
    custom=True,
    proj4def="""+proj=stere +lat_0=-90 +lat_ts=-71 +lon_0=0 +k=1
        +x_0=0 +y_0=0 +datum=WGS84 +units=m +no_defs""",
    bounds=[[-3174450, -2816050], [2867175, 2406325]],
)

MOA = WMSLayer(
    attribution="""
        U.S. Geological Survey (USGS), British Antarctic Survey (BAS),
        National Aeronautics and Space Administration (NASA)
        """,
    layers="MOA_125_HP1_090_230",
    format="image/png",
    transparent=False,
    opacity=0.5,
    url="https://nimbus.cr.usgs.gov/arcgis/services/Antarctica/USGS_EROS_Antarctica_Reference/MapServer/WmsServer",
    crs=MOA3031,
)
m2.add(MOA)

m2

## Custom Projections

Chances are that if you are reading this you want to use a custom projection for your map. This is now supported via [Proj4Leaflet](https://github.com/kartena/Proj4Leaflet). The only thing you need to do is to declare the properties of your projection and Leaflet will do the rest.

Keep in mind that this feature is for predefined projections, so if you use a backend as GeoServer, MapServer etc, they can do the reprojection for you and you can use that projection to show your layers in a map. **ipyleaflet won't reproject your layer on the fly**

In [ ]:
# We're going to use https://georepository.com/crs_2163/US-National-Atlas-Equal-Area.html
my_projection = {
    "name": "EPSG:2163",
    "custom": True,  # This is important, it tells ipyleaflet that this projection is not on the predefined ones.
    "proj4def": "+proj=laea +lat_0=45 +lon_0=-100 +x_0=0 +y_0=0 +a=6370997 +b=6370997 +units=m +no_defs",
    "origin": [-2215235.82, 420644.41],
    "bounds": [[-8046094.81, 1819060.18], [988364.71, 3511186.72]],
    "resolutions": [8192.0, 4096.0, 2048.0, 1024.0, 512.0, 256.0],
}


wms = WMSLayer(
    url="https://ahocevar.com/geoserver/wms",
    layers="ne:NE1_HR_LC_SR_W_DR",
    format="image/png",
    transparent=True,
    min_zoom=0,
    attribution="ahocevar geospatial",
    crs=my_projection,  # I'm asking this WMS service to reproject the tile layer using EPSG:2163
)

m3 = Map(center=(40, -104), zoom=0, layers=(wms,), crs=my_projection)


dc3 = DrawControl(marker={"shapeOptions": {"color": "#0000FF"}})
dc3.on_draw(handle_draw)

m3.add(dc3)

m3